<div >
<img src = "../banner.jpg" />
</div>

# Missclasification Rates

In [ ]:
#Cargar librerías 
require(pacman)
p_load(tidyverse)
set.seed(1011)

In [ ]:
#Leer los datos 
credit <- readRDS(url("https://github.com/ignaciomsarmiento/datasets/blob/main/credit_class.rds?raw=true"))
head(credit)

In [ ]:
table(credit$foreign)

In [ ]:
default<-credit$Default  #defino ahora va a servir despues
#mutacion de factores
credit<-credit %>% mutate(Default=factor(Default,levels=c(0,1),labels=c("No","Si")),
                          history=factor(history,levels=c("good","poor","terrible"),labels=c("buena","mala","terrible")),
                          foreign=factor(foreign,levels=c("foreign","german"),labels=c("extranjero","aleman")),
                          purpose=factor(purpose,levels=c("newcar","usedcar","goods/repair","edu", "biz" ),labels=c("auto_nuevo","auto_usado","bienes","educacion","negocios")))         


In [ ]:
head(credit)

In [ ]:
table(credit$foreign)

In [ ]:
## plot a mosaic
plot(Default ~ history, data=credit, col=c(8,2), ylab="Default") ## surprise!
## the dangers of choice-based sampling!  

In [ ]:
mylogit <- glm(Default~., data = credit, family = "binomial")
summary(mylogit,type="text")

In [ ]:
pred<-predict(mylogit,newdata = credit, type = "response")
summary(pred)


In [ ]:
## what are our misclassification rates?
rule <- 1/2 # Bayes Rule

sum( (pred>rule)[default==0] )/sum(pred>rule) ## false positive rate
sum( (pred<rule)[default==1] )/sum(pred<rule) ## false negative rate

sum( (pred>rule)[default==1] )/sum(default==1) ## sensitivity : Tasa Verdaderos Positivos
sum( (pred<rule)[default==0] )/sum(default==0) ## specificity: Tasa de Verdaderos Negativos

prop.table(table(default))
## what are our misclassification rates?
rule <- 1/5 
sum( (pred>rule)[default==0] )/sum(pred>rule) ## false positive rate
sum( (pred<rule)[default==1] )/sum(pred<rule) ## false negative rate

sum( (pred>rule)[default==1] )/sum(default==1) ## sensitivity: Tasa Verdaderos Positivos
sum( (pred<rule)[default==0] )/sum(default==0) ## specificity: Tasa de Verdaderos Negativos


In [ ]:
## plot the ROC curve for classification of y with p
roc <- function(p,y, ...){
  y <- factor(y)
  n <- length(p)
  p <- as.vector(p)
  Q <- p > matrix(rep(seq(0,1,length=100),n),ncol=100,byrow=TRUE)
  specificity <- colMeans(!Q[y==levels(y)[1],])
  sensitivity <- colMeans(Q[y==levels(y)[2],])
  plot(1-specificity, sensitivity, type="l", ...)
  abline(a=0,b=1,lty=2,col=8)
}



In [ ]:

roc(p=pred, y=default, bty="n")
## our 1/5 rule cutoff
points(x= 1-mean((pred<.2)[default==0]), 
       y=mean((pred>.2)[default==1]), 
       cex=4, pch=20, col='red') 
## a standard `max prob' (p=.5) rule
points(x= 1-mean((pred<.5)[default==0]), 
       y=mean((pred>.5)[default==1]), 
       cex=4, pch=20, col='blue') 
legend("bottomright",fill=c("red","blue"),
       legend=c("p=1/5","p=1/2"),bty="n",title="cutoff")


## Aside: Dummy Vars

In [ ]:
p_load("caret")
dmy <- dummyVars(" ~ .", data = credit)
head(dmy)

In [ ]:
credit <- data.frame(predict(dmy, newdata = credit))

In [ ]:
head(credit)

## Out of sample prediction

In [ ]:
credit<- credit  %>% mutate(Default=factor(Default.Si,levels=c(0,1),labels=c("No","Si")))

In [ ]:
inTrain <- createDataPartition(
  y = credit$Default.Si,## La variable dependiente u objetivo 
  p = .7, ## Usamos 70%  de los datos en el conjunto de entrenamiento 
  list = FALSE)


train <- credit[ inTrain,]
test  <- credit[-inTrain,]

In [ ]:
head(train)

### Logit

In [ ]:
fiveStats <- function(...) c(twoClassSummary(...), defaultSummary(...)) 



ctrl<- trainControl(method = "cv",
                    number = 5,
                    summaryFunction = fiveStats, 
                    classProbs = TRUE,
                    verbose=FALSE,
                    savePredictions = T)


mylogit_caret <- train(Default.Si~duration+amount+installment+age+
                       history.buena+history.mala+history.terrible+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+purpose.negocios+
                       foreign.extranjero+foreign.aleman+
                       rent.NA+rent.FALSE+rent.TRUE, 
                       data = train, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'ROC')




mylogit_caret


In [ ]:

mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+history.terrible+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+purpose.negocios+
                       foreign.extranjero+foreign.aleman+
                       rent.NA+rent.FALSE+rent.TRUE, 
                       data = train, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'ROC')




mylogit_caret


In [ ]:

mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = train, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'ROC')




mylogit_caret

In [ ]:
predictTest_logit <- data.frame(
  obs = test$Default,                                    ## observed class labels
  predict(mylogit_caret, newdata = test, type = "prob"),         ## predicted class probabilities
  pred = predict(mylogit_caret, newdata = test, type = "raw")    ## predicted class labels
)


In [ ]:
head(predictTest_logit)

In [ ]:
twoClassSummary(data = predictTest_logit, lev = levels(predictTest_logit$obs))

In [ ]:
prSummary(data = predictTest_logit, lev = levels(predictTest_logit$obs))

### LDA

#### By Hand

In [ ]:
p1<-sum(train$Default.Si)/dim(train)[1]
p1
mu1<-mean(train$duration[train$Default.Si==1])
mu1
mu0<-mean(train$duration[train$Default.Si==0])
mu0

g1<-sum((train$duration[train$Default.Si==1]-mu1)^2)
g0<-sum((train$duration[train$Default.Si==0]-mu0)^2)


sigma<-sqrt((g1+g0)/(dim(train)[1]-2))


f1<-dnorm(test$duration,mean=mu1,sd=sigma)
f0<-dnorm(test$duration,mean=mu0,sd=sigma)

In [ ]:
p_load("MASS")     # LDA
lda_simple <- lda(Default.Si~duration, data = train)
lda_simple_pred<-predict(lda_simple,test)
names(lda_simple_pred)
posteriors<-data.frame(lda_simple_pred$posterior)
posteriors$hand<-f1*p1/(f1*p1+f0*(1-p1))
head(posteriors)

#### Caret

In [ ]:
lda_fit = train(Default~duration+amount+installment+age, 
                data=train, 
                method="lda",
                trControl = ctrl,
                metric = 'ROC')

lda_fit

In [ ]:
predictTest_lda <- data.frame(
  obs = test$Default,                                    ## observed class labels
  predict(lda_fit, newdata = test, type = "prob"),         ## predicted class probabilities
  pred = predict(lda_fit, newdata = test, type = "raw")    ## predicted class labels
)



In [ ]:
head(predictTest_lda)

In [ ]:
twoClassSummary(data = predictTest_lda, lev = levels(predictTest_lda$obs))


In [ ]:
prSummary(data = predictTest_lda, lev = levels(predictTest_lda$obs))

 ### ROC performance

#### In-sample

In [ ]:
p_load("MLeval")

res_lda_fit <- evalm(lda_fit,plots="r")


In [ ]:
res_insample_all <- evalm(list(mylogit_caret,lda_fit),gnames=c('Logit','LDA'),plots='r')

#### Out-of-sample

In [ ]:
res_outsample_logit <- evalm(predictTest_logit,plots='r')

In [ ]:
res_outsample_lda <- evalm(predictTest_lda,plots='r')